In [1]:
import sys
import os
import subprocess
from six import string_types
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
import tensorflow as tf
from skimage import io
from scipy import ndimage
from IPython.display import display
import threading
%matplotlib inline

In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

Using TensorFlow backend.


In [3]:
Q_LOAD_BATCH = 20

Data preprocessing

In [15]:
class BasicBatchStreamer(object):
    
    def __init__(self, sess, batch_size, data_root="data", label_csv="train_v2.csv", jpeg_dir="train-jpg"):
        self.data_root = data_root
        self.label_csv = os.path.join(data_root, label_csv)
        self.jpeg_dir = os.path.join(data_root, jpeg_dir)
        assert os.path.exists(data_root)
        assert os.path.exists(self.label_csv)
        assert os.path.exists(self.jpeg_dir)
        self.y, self.file_names = self._process_labels()
        
        self.session = sess
        self.queue_input_chip = tf.placeholder(tf.float32, shape=(Q_LOAD_BATCH, 256, 256, 3))
        self.queue_input_label = tf.placeholder(tf.float32, shape=[Q_LOAD_BATCH, self.y.shape[1]])
        self.queue = tf.FIFOQueue(capacity=1000, dtypes=[tf.float32, tf.float32], shapes=[[256, 256, 3], [self.y.shape[1]]])
        self.enqueue_op = self.queue.enqueue_many([self.queue_input_chip, self.queue_input_label])
        self.dequeue_op = self.queue.dequeue()
        
        
        min = batch_size * 2
        capacity = min + batch_size * 3
        self.batch_X, self.batch_y = tf.train.shuffle_batch(self.dequeue_op, batch_size=batch_size, capacity=capacity, min_after_dequeue=min)
        
        self.cur_file_idx = 0
        self.enqueue_thread = threading.Thread(target=self._enqueue_daemon, args=[])
        self.enqueue_thread.isDaemon()
        self.enqueue_thread.daemon = True
        self.enqueue_thread.start()
        
        self.coord = tf.train.Coordinator()
        self.threads = tf.train.start_queue_runners(coord=self.coord, sess=self.session)
        
    def _process_labels(self):
        labels_df = pd.read_csv(self.label_csv)
        
        # Find all availabe tags and map to index
        label_list = []
        for tag_str in labels_df.tags.values:
            labels = tag_str.split(" ")
            for label in labels:
                if label not in label_list:
                    label_list.append(label)
        label_map = {}
        for i in range(len(label_list)):
            label_map[label_list[i]] = i
        print("All available labels: ", label_map)
        
        # create one hot vectors
        y = np.zeros((labels_df.shape[0], len(label_list)))
        file_names = []
        for i in range(labels_df.shape[0]):
            labels = labels_df.tags.values[i].split(" ")
            file_names.append(labels_df.image_name.values[i])
            for label in labels:
                y[i, label_map[label]] = 1
        return y, file_names
        
    def _enqueue_daemon(self):
        while True:
            X = np.ndarray([Q_LOAD_BATCH, 256, 256, 3])
            y = np.ndarray([Q_LOAD_BATCH, self.y.shape[1]])
            for i in range(Q_LOAD_BATCH):
                file_path = os.path.join(self.jpeg_dir, self.file_names[self.cur_file_idx] + ".jpg")
                X[i, :, :, :] = io.imread(file_path)
                y[i, :] = self.y[i]
                self.cur_file_idx += 1
                self.cur_file_idx %= self.y.shape[0]
            sess.run(self.enqueue_op, feed_dict={self.queue_input_chip: X,
                                            self.queue_input_label: y})
    def get_batch(self):
        batch_X, batch_y = sess.run([self.batch_X, self.batch_y])
        return batch_X, batch_y
    
    def close(self):
        self.session.run(self.queue.close(cancel_pending_enqueues=True))
        self.coord.request_stop()
        self.coord.join(self.threads)

In [ ]:
sess = tf.Session()
data_loader = BasicBatchStreamer(sess, 128)
for i in range(100):
    X, y = data_loader.get_batch()
    print(i)

('All available labels: ', {'slash_burn': 8, 'clear': 3, 'blooming': 14, 'primary': 1, 'cloudy': 9, 'conventional_mine': 11, 'water': 4, 'haze': 0, 'cultivation': 7, 'partly_cloudy': 10, 'artisinal_mine': 13, 'habitation': 5, 'bare_ground': 12, 'blow_down': 16, 'agriculture': 2, 'road': 6, 'selective_logging': 15})
0
1
2
3
4
5
6
7
8
9
10


In [5]:
sPLANET_KAGGLE_ROOT = os.path.abspath("data")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

In [9]:
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()
print(labels_df.shape)

(40479, 2)


In [7]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [8]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
labels_df.head()

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [3]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of bcbd6486424b2319ff4ef7d526e38f63 so we will re-download the data.
  180224/87910968 [..............................] - ETA: 348s

KeyboardInterrupt: 